In this notebook, NCHS Multiple Cause-of-Death mortality data recodes used in 2016-2022 data are compared, specifically: 39_cause_recode, 113_cause_recode, and 358_cause_recode. The primary key with which these "underlying cause of death" recodes are collated is the singular ICD10 code provided for each death record.

In [8]:
# Import libraries
import pandas as pd
import os

# Set working directory
os.chdir('D:/d4ph/nchs-death')

# Import cleaned mortality data from each year (2016-2022)
mort_codes_2016 = pd.read_csv('mort2016_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2017 = pd.read_csv('mort2017_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2018 = pd.read_csv('mort2018_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2019 = pd.read_csv('mort2019_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2020 = pd.read_csv('mort2020_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2021 = pd.read_csv('mort2021_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes_2022 = pd.read_csv('mort2022_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])

In [9]:
# Concatenate all years into one dataframe
mort_codes_all = pd.concat([mort_codes_2016, mort_codes_2017, mort_codes_2018, mort_codes_2019, mort_codes_2020, mort_codes_2021, mort_codes_2022])

# Keep only unique rows
mort_codes_all = mort_codes_all.drop_duplicates()

# Export to CSV
mort_codes_all.to_csv('mort_codes_all.csv', index=False)

In [ ]:
# Print shape of mort_codes_all
print(mort_codes_all.shape)

# Print first 5 rows of mort_codes_all
print(mort_codes_all.head())

(5269, 4)
  icd10  358_cause_recode  113_cause_recode  39_cause_recode
0  K566               293               111               37
1   I64               238                70               24
2  C189                81                23                6
3  K767               301               111               37
4  E785               173               111               37


In [1]:
import pandas as pd
from dash import Dash, dcc, html, dash_table, Input, Output
import plotly.graph_objects as go
from dash.exceptions import PreventUpdate

# Load the CSV data
file_path = 'mort_codes_all.csv'
mort_data = pd.read_csv("mort_codes_all.csv")

# Initialize the Dash app
app = Dash(__name__)

# Define the layout
app.layout = html.Div([
    html.H1("ICD-10 Recode Mapping Explorer"),
    
    # Dropdown for selecting recode map or ICD-10 code
    html.Div([
        html.Label("Select a Recode Map or ICD-10 Code:"),
        dcc.Dropdown(
            id='selection-dropdown',
            options=[
                {'label': 'Search by 358 Cause Recode', 'value': '358_cause_recode'},
                {'label': 'Search by 113 Cause Recode', 'value': '113_cause_recode'},
                {'label': 'Search by 39 Cause Recode', 'value': '39_cause_recode'},
                {'label': 'Search by ICD-10 Code', 'value': 'icd10'}
            ],
            placeholder="Select or search..."
        ),
        dcc.Input(
            id='icd10-input',
            type='text',
            placeholder='Enter ICD-10 Code (e.g., K566)',
            style={'marginTop': '10px', 'display': 'none'}  # Hidden by default
        )
    ], style={'width': '50%', 'margin': '20px auto'}),
    
    # DataTable for displaying results
    html.Div([
        dash_table.DataTable(
            id='mapping-table',
            columns=[{"name": col, "id": col} for col in mort_data.columns],
            style_table={'overflowX': 'auto'},
            style_cell={'textAlign': 'left', 'minWidth': '100px', 'width': '100px', 'maxWidth': '100px'},
        )
    ], style={'margin': '20px auto', 'width': '80%'}),
    
    # Sankey diagram for visualization
    html.Div([
        dcc.Graph(id='sankey-diagram')
    ], style={'margin': '20px auto', 'width': '80%'})
])

# Callbacks for interactivity
from dash.exceptions import PreventUpdate

@app.callback(
    [Output('icd10-input', 'style'), Output('mapping-table', 'data')],
    [Input('selection-dropdown', 'value'), Input('icd10-input', 'value')]
)
def update_table(selection, lookup_value):
    # Show/hide the ICD-10 input field based on the dropdown selection
    icd10_style = {'marginTop': '10px', 'display': 'none', 'width': '75%'}
    filtered_data = mort_data

    if selection:  # If a selection is made
        icd10_style = {'marginTop': '10px', 'display': 'block', 'width': '75%'}
        if lookup_value:
            try:
                if selection == 'icd10':
                    # Filter by ICD-10 code
                    filtered_data = mort_data[mort_data['icd10'] == lookup_value]
                elif selection in ['358_cause_recode', '113_cause_recode', '39_cause_recode']:
                    # Filter by recode value
                    filtered_data = mort_data[mort_data[selection] == int(lookup_value)]
            except ValueError:
                # If a ValueError occurs, return an empty table with a warning message
                return icd10_style, [{"icd10": "Enter Valid Code"}]
    
    return icd10_style, filtered_data.to_dict('records')


@app.callback(
    Output('sankey-diagram', 'figure'),
    [Input('selection-dropdown', 'value'), Input('icd10-input', 'value')]
)
def update_sankey(selection, lookup_value):
    if not selection or not lookup_value:
        return go.Figure()

    try:
        # Filter the data based on the selection and lookup value
        if selection == 'icd10':
            filtered_data = mort_data[mort_data['icd10'] == lookup_value]
        else:
            filtered_data = mort_data[mort_data[selection] == int(lookup_value)]
    except ValueError:
        # Return an empty figure if the input is invalid
        return go.Figure()

    if filtered_data.empty:
        return go.Figure()

    # Prepare labels and mappings
    icd10_labels = filtered_data['icd10'].tolist()
    recode_358_labels = filtered_data['358_cause_recode'].astype(str).tolist()
    recode_113_labels = filtered_data['113_cause_recode'].astype(str).tolist()
    recode_39_labels = filtered_data['39_cause_recode'].astype(str).tolist()

    labels = list(set(icd10_labels + recode_358_labels + recode_113_labels + recode_39_labels))
    label_to_index = {label: i for i, label in enumerate(labels)}

    sources, targets, values = [], [], []

    # Map ICD-10 → 358 Cause Recode
    for icd10, recode_358 in zip(icd10_labels, recode_358_labels):
        sources.append(label_to_index[icd10])
        targets.append(label_to_index[recode_358])
        values.append(1)

    # Map 358 → 113 Cause Recode
    for recode_358, recode_113 in zip(recode_358_labels, recode_113_labels):
        sources.append(label_to_index[recode_358])
        targets.append(label_to_index[recode_113])
        values.append(1)

    # Map 113 → 39 Cause Recode
    for recode_113, recode_39 in zip(recode_113_labels, recode_39_labels):
        sources.append(label_to_index[recode_113])
        targets.append(label_to_index[recode_39])
        values.append(1)

    # Create the Sankey diagram
    sankey_fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
        )
    )])

    sankey_fig.update_layout(
        title_text=f"Mapping for {selection} Value: {lookup_value}",
        font_size=10
    )
    return sankey_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


The meaning of each code can be found at https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/DVS/mortality/2022-Mortality-Public-Use-File-Documentation.pdf, but I'm going to map the names of these recode schema.

In [28]:
# Create names map of 358 cause recode
recode_358_map = pd.read_csv('./recodeMaps/358recodeMap.csv')
recode_358_dict = dict(zip(recode_358_map['key'], recode_358_map['value']))

# Create names map of 113 cause recode
recode_113_map = pd.read_csv('./recodeMaps/113recodeMap.csv')
recode_113_dict = dict(zip(recode_113_map['key'], recode_113_map['value']))

# Create names map of 39 cause recode
recode_39_map = pd.read_csv('./recodeMaps/39recodeMap.csv')
recode_39_dict = dict(zip(recode_39_map['key'], recode_39_map['value']))

Now that we have our value maps, let's create a dataframe with total number of U.S. deaths in 2022 by icd10, 358_cause_recode, 113_cause_recode, and 39_cause_recode.

In [10]:
# Load the CSV data
mort_2022 = pd.read_csv('mort2022_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
mort_codes = pd.read_csv("mort_codes_all.csv")

# Add column in mort_codes called 'count' that counts the number of rows in mort_2022 with a matching icd10 code
mort_codes['count'] = mort_codes['icd10'].map(mort_2022['icd10'].value_counts())

# Print first 5 rows of mort_codes_all
print(mort_codes.head())

# Print shape of mort_codes_all
print("shape of mort_codes:", mort_codes.shape)
print("shape of mort_2022:", mort_2022.shape)

# Print sum of count column
print("sum of mort_codes:", mort_codes['count'].sum())
print("sum of mort_2022:", mort_2022.shape[0])

  icd10  358_cause_recode  113_cause_recode  39_cause_recode    count
0  K566               293               111               37   6137.0
1   I64               238                70               24  47111.0
2  C189                81                23                6  35935.0
3  K767               301               111               37    509.0
4  E785               173               111               37  15615.0
shape of mort_codes: (5269, 5)
shape of mort_2022: (3289569, 4)
sum of mort_codes: 3289581.0
sum of mort_2022: 3289569


There is a slight discrepancy in the respective sums of our base 2022 mortality data and unique codes, but its 99.9996% alignment is sufficient for our present purposes.

Now, I'll add additional columns withour mapped schema names. 

In [23]:
#print first 5 rows of mort_2022
print(mort_codes.head())

  icd10  358_cause_recode  113_cause_recode  39_cause_recode    count  \
0  K566               293               111               37   6137.0   
1   I64               238                70               24  47111.0   
2  C189                81                23                6  35935.0   
3  K767               301               111               37    509.0   
4  E785               173               111               37  15615.0   

                               358_cause_recode_name  \
0  Vascular disorders and obstruction of intestin...   
1  Stroke, not specified as hemorrhage or infarct...   
2                                        Colon (C18)   
3         Other diseases of liver (K7l-K72, K75-K76)   
4      Other metabolic disorders (E70-E83, E85, E88)   

                               113_cause_recode_name  
0  All other diseases (Residual) (D65-E07,E15-E34...  
1                 Cerebrovascular diseases (I60-I69)  
2  Malignant neoplasms of colon, rectum and anus ...  
3  A

In [31]:
# Add columns to mort_codes for 358, 113, and 39 cause recode names
mort_codes['358_cause_recode_name'] = mort_codes['358_cause_recode'].map(recode_358_dict)
mort_codes['113_cause_recode_name'] = mort_codes['113_cause_recode'].map(recode_113_dict)
mort_codes['39_cause_recode_name'] = mort_codes['39_cause_recode'].map(recode_39_dict)

# Print first 5 rows of mort_codes
print(mort_codes.head())

# Export to CSV
mort_codes.to_csv('./outputs/usMortality_2022.csv', index=False)

  icd10  358_cause_recode  113_cause_recode  39_cause_recode    count  \
0  K566               293               111               37   6137.0   
1   I64               238                70               24  47111.0   
2  C189                81                23                6  35935.0   
3  K767               301               111               37    509.0   
4  E785               173               111               37  15615.0   

                               358_cause_recode_name  \
0  Vascular disorders and obstruction of intestin...   
1  Stroke, not specified as hemorrhage or infarct...   
2                                        Colon (C18)   
3         Other diseases of liver (K7l-K72, K75-K76)   
4      Other metabolic disorders (E70-E83, E85, E88)   

                               113_cause_recode_name  \
0  All other diseases (Residual) (D65-E07,E15-E34...   
1                 Cerebrovascular diseases (I60-I69)   
2  Malignant neoplasms of colon, rectum and anus ...   


Nice. Ok, now to repeat the process for 2016-2021, I'll create a function to loop through the years of our data and output a corresponding mortality file.

In [ ]:
def process_year(year):
    # Load the CSV data for the given year
    mort_year = pd.read_csv(f'mort{year}_cleaned.csv', usecols=['icd10', '39_cause_recode', '113_cause_recode', '358_cause_recode'])
    
    # Reload mort_codes
    mort_codes = pd.read_csv("mort_codes_all.csv")

    # Add column in mort_codes called 'count' that counts the number of rows in mort_year with a matching icd10 code
    mort_codes['count'] = mort_codes['icd10'].map(mort_year['icd10'].value_counts())
    
    # Add columns to mort_codes for 358, 113, and 39 cause recode names
    mort_codes['358_cause_recode_name'] = mort_codes['358_cause_recode'].map(recode_358_dict)
    mort_codes['113_cause_recode_name'] = mort_codes['113_cause_recode'].map(recode_113_dict)
    mort_codes['39_cause_recode_name'] = mort_codes['39_cause_recode'].map(recode_39_dict)
    
    # Export to CSV
    mort_codes.to_csv(f'./outputs/usMortality_{year}.csv', index=False)

# Process each year from 2016 to 2021
for year in range(2016, 2022):
    process_year(year)

Several analyses will follow, but for now, I will leverage the fantastic work of data sculptor Vasco Asturiano (https://github.com/vasturiano/sunburst-chart) and put the 2022 data into a zoomable sunburst chart.

To do this, I need to convert my dataframe into the requisite .json file for this visualization.

In [ ]:
import pandas as pd
import json

# Load the data
data = pd.read_csv('./outputs/usMortality_2022.csv')

# Remove rows with NaN or zero in the 'count' column
data = data.dropna(subset=['count'])  # Remove rows with NaN in 'count'
data = data[data['count'] > 0]        # Remove rows with zero or negative counts

# Function to build the hierarchical structure
def build_hierarchy(data):
    hierarchy = []
    grouped_39 = data.groupby('39_cause_recode_name')

    for name_39, group_39 in grouped_39:
        node_39 = {
            "name": f"{name_39}",
            "children": []
        }
        grouped_113 = group_39.groupby('113_cause_recode_name')

        for name_113, group_113 in grouped_113:
            node_113 = {
                "name": f"{name_113}",
                "children": []
            }
            grouped_358 = group_113.groupby('358_cause_recode_name')

            for name_358, group_358 in grouped_358:
                node_358 = {
                    "name": f"{name_358}",
                    "children": []
                }
                for _, row in group_358.iterrows():
                    node_358["children"].append({
                        "name": row["icd10"],
                        "size": row["count"]
                    })
                node_113["children"].append(node_358)
            node_39["children"].append(node_113)
        hierarchy.append(node_39)
    return hierarchy

# Build the hierarchy
hierarchy_data = {
    "name": "2022 U.S. Deaths",
    "children": build_hierarchy(data)
}

# Save to a JSON file
output_file = './outputs/usMortality_2022b.json'
with open(output_file, 'w') as f:
    json.dump(hierarchy_data, f, indent=4)

print(f"Cleaned JSON data has been saved to {output_file}")

Cleaned JSON data has been saved to ./outputs/usMortality_2022b.json


I plugged it in and... TADAAAAA!!!

https://dataforpublichealth.digital/d3/sunburst/nchs-mortality-2022/index.html